In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import keras
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [7]:
data=pd.read_excel('/home/tar-tt085-aryan/Downloads/vessel_detection.xlsx')

In [8]:
data.head()

,Vessel Type,Latitude,Longitude,Time,Speed (knots),Course (Degrees),X (Cartesian),Y (Cartesian),Polar Distance (NM),Polar Angle (Degrees),...,Interaction,Heading,Vessel Condition,AIS Signal,Communication,Turning Rate,Dwell Time (hours),Lightning,Anomalous Behavior,Friend or Foe
0,Fishing Vessel,34.0522,-118.2437,08:30:00,8.0,180.0,100.0,100.0,10.0,180.0,...,2.0,180.0,7.0,ON,YES,5.0,2.0,OFF,NO,Friend
1,Tanker,40.7128,-74.0060,22:15:00,20.0,270.0,50.0,200.0,15.0,270.0,...,3.0,270.0,8.0,ON,YES,3.0,1.5,ON,YES,Friend
2,Container Ship,37.7749,-122.4194,11:45:00,25.0,90.0,150.0,50.0,20.0,90.0,...,1.0,90.0,6.0,OFF,NO,4.0,0.8,OFF,YES,Foe
3,Passenger Ship,51.5074,-0.1278,03:20:00,20.0,45.0,200.0,150.0,25.0,45.0,...,4.0,45.0,9.0,ON,YES,6.0,2.5,ON,NO,Friend
4,Bulk Carrier,37.7749,-122.4194,14:40:00,12.0,135.0,120.0,100.0,18.0,135.0,...,2.0,135.0,5.0,ON,YES,2.0,0.3,OFF,YES,Foe


In [9]:
data.shape

(99, 21)

In [10]:
X = data.drop('Friend or Foe', axis=1) 
y = data['Friend or Foe']  


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Vessel Type                99 non-null     object 
 1   Latitude                   98 non-null     float64
 2   Longitude                  98 non-null     float64
 3   Time                       98 non-null     object 
 4   Speed (knots)              97 non-null     float64
 5   Course (Degrees)           97 non-null     float64
 6   X (Cartesian)              97 non-null     float64
 7   Y (Cartesian)              97 non-null     float64
 8   Polar Distance (NM)        97 non-null     float64
 9   Polar Angle (Degrees)      97 non-null     float64
 10  Stoppage Duration (hours)  97 non-null     float64
 11  Interaction                97 non-null     float64
 12  Heading                    97 non-null     float64
 13  Vessel Condition           97 non-null     float64
 

In [22]:
label_encoder = LabelEncoder()
X['Vessel Type'] = label_encoder.fit_transform(X['Vessel Type'])
X['Vessel Condition'] = label_encoder.fit_transform(X['Vessel Condition'])
X['AIS Signal'] = label_encoder.fit_transform(X['AIS Signal'])
X['Communication'] = label_encoder.fit_transform(X['Communication'])
X['Lightning'] = label_encoder.fit_transform(X['Lightning'])
X['Anomalous Behavior'] = label_encoder.fit_transform(X['Anomalous Behavior'])


In [23]:
label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y)

print(y_encoded)


[1 1 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0
 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 2 0 1 0 2 0
 1 0 1 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1]


In [25]:
data['Friend or Foe']  

0     Friend
1     Friend
2        Foe
3     Friend
4        Foe
       ...  
94       Foe
95    Friend
96       Foe
97       Foe
98    Friend
Name: Friend or Foe, Length: 99, dtype: object

In [26]:
X['Vessel Type']

0     2
1     4
2     1
3     3
4     0
     ..
94    1
95    3
96    0
97    2
98    4
Name: Vessel Type, Length: 99, dtype: int64

In [29]:
def time_con(time_str):
    hours, minutes = map(int, time_str.split(':'))
    
    numerical_time = hours + minutes / 60
    
    return numerical_time
time_

In [27]:
scalar=StandardScaler()

In [28]:
x_scale= scalar.fit_transform(X)

TypeError: float() argument must be a string or a real number, not 'datetime.time'

In [31]:
x_train,x_test,y_train,y_test = train_test_split(X,y_encoded,random_state=42)

# hyperparameter

In [32]:
def build_hp(hp):
    model = keras.Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=512, step=32),
                    activation='relu', input_dim=x_train.shape[1]))
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                        activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model  

# initilization

In [33]:
x_train.shape[1]

13

In [34]:
tuner = RandomSearch(
    build_hp,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='aryanv_dir',
    project_name='my_project',
    max_consecutive_failed_trials=10,  # Increase this value
)


Reloading Tuner from aryanv_dir/my_project/tuner0.json


In [37]:
tuner.search(x_train, y_train, epochs=10, validation_split=0.2)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

model.fit(x_train, y_train, epochs=100, validation_split=0.2)

eval_result = model.evaluate(x_test, y_test)
print("Test accuracy:", eval_result[1])

Epoch 1/100


/home/tar-tt085-aryan/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.4681 - loss: 14.8831 - val_accuracy: 0.4667 - val_loss: 37.3610
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5979 - loss: 18.1701 - val_accuracy: 0.5333 - val_loss: 24.2430
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4146 - loss: 26.4373 - val_accuracy: 0.5333 - val_loss: 4.9612
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5007 - loss: 5.6437 - val_accuracy: 0.4667 - val_loss: 18.7092
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5437 - loss: 12.0471 - val_accuracy: 0.4667 - val_loss: 10.4160
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7076 - loss: 4.3166 - val_accuracy: 0.5333 - val_loss: 6.2553
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4354 - loss: 8.1371 - val_accuracy: 0.6667 - val_loss: 1.3604
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4569 - loss: 2.5932 - val_accuracy: 0.4667 - val_loss: 6.6